In [1]:
import geemap
import ee
import shapefile

In [2]:
#ee.Authenticate()

In [3]:
# Initialize Earth Engine
geemap.ee_initialize()

In [4]:
# Create an interactive map
Map = geemap.Map()

# Add a basemap
Map.add_basemap('HYBRID')

In [5]:
sf = shapefile.Reader('Uttarakhand/Uttarakhand.shp')
feature = sf.shapeRecords()[0]  # Assuming there is only one feature in the shapefile
aoi = ee.Geometry.Polygon(feature.shape.__geo_interface__['coordinates'])
#print(aoi)

In [6]:
# Load Global Forest Change data
gfc = ee.Image("UMD/hansen/global_forest_change_2020_v1_8").clip(aoi)

# Select bands
treecover2000 = gfc.select('treecover2000')
loss = gfc.select('loss')
gain = gfc.select('gain')

In [7]:
# Create a mask for forest areas in 2000
forest2000 = treecover2000.gte(30)

# Create a mask for forest loss from 2001 to 2020
loss01_20 = loss.eq(1)

# Create a mask for forest gain from 2001 to 2020
gain01_20 = gain.eq(1)

# Create a mask for afforestation from 2001 to 2020
afforestation01_20 = gain01_20.And(forest2000.Not())

# Create a mask for deforestation from 2001 to 2020
deforestation01_20 = loss01_20.And(forest2000)

In [8]:
# Calculate carbon sequestration potential
carbon_sequestration_afforestation = afforestation01_20.multiply(10)  
carbon_sequestration_deforestation = deforestation01_20.multiply(-10)  

In [9]:
# Calculate net carbon sequestration (afforestation - deforestation)
net_carbon_sequestration = carbon_sequestration_afforestation.add(carbon_sequestration_deforestation)

In [10]:
# Define a color palette for the carbon sequestration map
carbon_palette = ['red', 'white', 'green']

# Add the aoi and net carbon sequestration layer to the map with the color palette
#Map.addLayer(aoi, {'color': 'blue'}, 'Study Area')
Map.addLayer(net_carbon_sequestration, {'min':-100, 'max': 100, 'palette': carbon_palette, 'opacity': 0.5}, 'Net Carbon Sequestration')

# Highlight Loss and Gain layers
Map.addLayer(loss01_20.mask(loss01_20), {'palette': 'FF0000'}, 'Loss')
Map.addLayer(gain01_20.mask(gain01_20), {'palette': '00FF00'}, 'Gain')

# Center the map on the specified latitude and longitude
Map.centerObject(aoi, 10)
legend_dict = { "Loss": "#FF0000", "Neutral": "#FF000000", "Gain": "#00FF00" }
Map.addLayerControl()
Map.add_legend(legend_dict=legend_dict)

# Display the map
Map

No such comm: 32904196dc24463f8da80fac26464593
No such comm: 32904196dc24463f8da80fac26464593
No such comm: 32904196dc24463f8da80fac26464593
No such comm: 32904196dc24463f8da80fac26464593
No such comm: 32904196dc24463f8da80fac26464593
No such comm: 32904196dc24463f8da80fac26464593
No such comm: 32904196dc24463f8da80fac26464593
No such comm: 32904196dc24463f8da80fac26464593
No such comm: 32904196dc24463f8da80fac26464593
No such comm: 32904196dc24463f8da80fac26464593


Map(center=[29.881332611704394, 77.9961229718262], controls=(WidgetControl(options=['position', 'transparent_b…

In [11]:
# Calculate the total and average net carbon sequestration
total_carbon_sequestration = net_carbon_sequestration.reduceRegion(reducer=ee.Reducer.sum(), geometry=aoi, scale=30)

# Calculate area of afforestation and deforestation
area_afforestation = gain01_20.reduceRegion(reducer=ee.Reducer.sum(), geometry=aoi, scale=30)
area_deforestation = loss01_20.reduceRegion(reducer=ee.Reducer.sum(), geometry=aoi, scale=30)

# Calculate annual rates of afforestation and deforestation
years = 22
afforestation_rate = area_afforestation.get("gain").getInfo() / (years)  
deforestation_rate = area_deforestation.get("loss").getInfo() / (years) 

# Print the calculated statistics
print("Total Carbon Sequestration:", total_carbon_sequestration.getInfo())
print("Area of Afforestation in m2:", area_afforestation.getInfo())
print("Area of Deforestation in m2:", area_deforestation.getInfo())
print("Afforestation Rate (m2/yr):", afforestation_rate)
print("Deforestation Rate (m2/yr):", deforestation_rate)

Total Carbon Sequestration: {'gain': 9355.764705882353}
Area of Afforestation in m2: {'gain': 2691}
Area of Deforestation in m2: {'loss': 16358.784313725491}
Afforestation Rate (m2/yr): 122.31818181818181
Deforestation Rate (m2/yr): 743.5811051693405
